In [4]:
import os
#char-rnn model generate chinese poem 
#dynamic rnn - 1 layer
#one hot encoding as word embedding method

class CHARRNN:
    def __init__(self,**param):
        self.rnn_size=param.get('rnn_size',128)
        self.num_layers=param.get('num_layers',2)
        self.model=param.get('model','LSTM')
        #self.batch_size=param.get('batch_size',64)
        self.length=param.get('lenword')
        self.sess=tf.Session()
        self.n_chunk=param.get('n_chunk')
        self.cell=tf.contrib.rnn.BasicLSTMCell(self.rnn_size,reuse=tf.get_variable_scope().reuse) 

        
    def _weights(self,shape,name='weights'):
        initializer=tf.random_normal_initializer(mean=0.,stddev=0.5)
        return tf.get_variable(shape=shape,initializer=initializer,name=name)
    
    def _bias(self,shape,name='bias'):
        initializer=tf.constant_initializer(0.)
        return tf.get_variable(shape=shape,initializer=initializer,name=name)
    
    def mask(self,n):
        return np.random.choice(n)
    
    def output(self,batch_size,reuse=False):

        with tf.variable_scope('output',reuse=reuse):
            self.W=self._weights([self.rnn_size,self.length])
            self.b=self._bias([self.length])
            with tf.variable_scope('embedding',reuse=reuse):
                embedding=self._weights([self.length,self.rnn_size],name='embed')
            inputs=tf.nn.embedding_lookup(embedding,self.x)
            self.initial_state=self.cell.zero_state(batch_size,tf.float32) 
            outputs,self.last_state=tf.nn.dynamic_rnn(self.cell,inputs,initial_state=self.initial_state)
            output=tf.reshape(outputs,[-1,self.rnn_size])
            self.logits=tf.matmul(output,self.W)+self.b
            self.probs=tf.nn.softmax(self.logits)
        return self.logits,self.probs,self.last_state
        
    
    def Train(self,x_batch,y_batch,batch_size):
       #with tf.name_scope('inputs'):
        self.x=tf.placeholder(tf.int32,[batch_size,None])
        self.y=tf.placeholder(tf.int32,[batch_size,None])    
        with tf.name_scope('train'):
            targets=tf.reshape(self.y,[-1])
            self.logits,self.probs,self.last_state=self.output(batch_size)
            loss=tf.contrib.legacy_seq2seq.sequence_loss_by_example([self.logits],[targets],[tf.ones_like(targets, dtype=tf.float32)])
            cost=tf.reduce_mean(loss)
            learning_rate = tf.Variable(0.0, trainable=False)
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), 5)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            train_op = optimizer.apply_gradients(zip(grads, tvars))
            
            saver=tf.train.Saver(tf.all_variables())
            init=tf.global_variables_initializer()
            self.sess.run(init)
            for epoch in range(50):
                self.sess.run(tf.assign(learning_rate, 0.002 * (0.97 ** epoch)))
                n=0
                for i in range(self.n_chunk):
                        #n=self.mask(self.n_chunk-1)
                    train_loss,_=self.sess.run([cost,train_op],feed_dict={self.x:x_batch[n],self.y:y_batch[n]})
                    n+=1
                    if i%600==0:
                        print('loss:{:4}'.format(train_loss))
                saver.save(self.sess,os.path.join(os.getcwd(), 'charrnn.ckpt'))
                            
    def to_word(self,weights,words):
        t=np.cumsum(weights)
        s=np.sum(weights)
        sample=int(np.searchsorted(t,np.random.rand(1)*s))
        return words[sample]
    
    def predict(self,words,word_num_map,batch_size,reuse=False):
        self.x=tf.placeholder(tf.int32,[batch_size,None])
        self.y=tf.placeholder(tf.int32,[batch_size,None])         
                
        with tf.name_scope('predict'):
            self.initial_state=self.cell.zero_state(batch_size,tf.float32)  
            gx=np.array([list(map(word_num_map.get,'['))])
            self.logits,self.probs,self.last_state=self.output(batch_size,reuse=reuse)
            init=tf.global_variables_initializer()
            self.sess.run(init)
            saver=tf.train.Saver()
            saver.restore(self.sess,os.path.join(os.getcwd(), 'charrnn.ckpt'))
            state_=self.sess.run(self.cell.zero_state(1,tf.float32)) 
            [probs_,state_]=self.sess.run([self.probs,self.last_state],feed_dict={self.x:gx,self.initial_state:state_})
            word=self.to_word(probs_,words)
            poem=''
            while word !=']':
                poem+=word
                gx=np.zeros((1,1))
                gx[0,0]=word_num_map[word]
            #with tf.control_dependencies([self.initial_state.assign(state_)]):
                [probs_,state_]=self.sess.run([self.probs,self.last_state],feed_dict={self.x:gx,self.initial_state:state_})
                word=self.to_word(probs_,words)
        return poem

In [2]:
import collections
import numpy as np
import tensorflow as tf

#data pre-process transfer poem word into int value according to word frequency

poetry_file='poetry.txt'
poetrys=[]
with open(poetry_file,'r',encoding='utf-8',) as f:
    for line in f:
        try:
            title,content=line.strip().split(':')
            content=content.replace(' ','')
            if '_' in content or '(' in content or '（' in content or '《' in content or '[' in content: 
                continue
            if len(content)<5 or len(content)>79:
                continue
            content='['+content+']'
            poetrys.append(content)
        except Exception as e:
            pass

poetrys=sorted(poetrys,key=lambda line: len(line))
print('诗词总数:',len(poetrys))

all_words=[]
for poetry in poetrys:
    all_words+=[word for word in poetry]
counter=collections.Counter(all_words)
count_pairs=sorted(counter.items(),key=lambda x:-x[-1])
words,_=zip(*count_pairs)

words=words[:len(words)]+(' ',)
word_nump_map=dict(zip(words,range(len(words))))

to_num=lambda word:word_nump_map.get(word,len(words))
poetrys_vector=[list(map(to_num,poetry)) for poetry in poetrys]

batch_size=64
n_chunk=len(poetrys_vector)//batch_size
x_batches=[]
y_batches=[]
for i in range(n_chunk):
    start_index=i*batch_size
    end_index=start_index+batch_size
    
    batches=poetrys_vector[start_index:end_index]
    length=max(map(len,batches))
    xdata=np.full((batch_size,length),word_nump_map[' '],np.int32)
    for row in range(batch_size):
        xdata[row,:len(batches[row])]=batches[row]
    ydata=np.copy(xdata)
    ydata[:,:-1]=xdata[:,1:]
    x_batches.append(xdata)
    y_batches.append(ydata)

print(x_batches)

诗词总数: 34646
[array([[   2,   28,  543,  104,  720,    1,    3, 6109, 6109, 6109],
       [   2,  649,   48,    9, 2138,    1,    3, 6109, 6109, 6109],
       [   2,  424, 4637, 2126, 1100,    1,    3, 6109, 6109, 6109],
       [   2,  345, 1145, 2601, 2660,    1,    3, 6109, 6109, 6109],
       [   2,  821,   10, 1366,  332,    1,    3, 6109, 6109, 6109],
       [   2, 1196,   57,  430,   34,  438,    1,    3, 6109, 6109],
       [   2,    5,   35,  140,  158,  105,    1,    3, 6109, 6109],
       [   2,  292, 1461,   18,   56,  298,    1,    3, 6109, 6109],
       [   2, 1139,  945, 1320,  522,  661,    1,    3, 6109, 6109],
       [   2, 1363,  104,   18,   83,  583,    1,    3, 6109, 6109],
       [   2,  649,   48,    9, 2138,  392,    1,    3, 6109, 6109],
       [   2,  168, 2319, 1625, 1075, 2319,    1,    3, 6109, 6109],
       [   2,   64,   28,   47,   64,  739,    1,    3, 6109, 6109],
       [   2, 1035,  327,   28,  513,  235,    1,    3, 6109, 6109],
       [   2,  270, 1

In [11]:
#train model
myrnn=CHARRNN(lenword=len(words)+1,n_chunk=n_chunk)
myrnn.Train(x_batches,y_batches,64)

Instructions for updating:
Please use tf.global_variables instead.
loss:8.93932819366455
loss:6.584232807159424
loss:5.928378582000732
loss:5.660006523132324
loss:5.374624729156494
loss:5.175785541534424
loss:4.9493489265441895
loss:4.765604019165039
loss:4.621252536773682
loss:4.493691444396973
loss:4.3825764656066895
loss:4.278059959411621
loss:4.1846466064453125
loss:4.101984024047852
loss:4.022896766662598
loss:3.9448719024658203
loss:3.871077060699463
loss:3.8040971755981445
loss:3.74234676361084
loss:3.684908390045166
loss:3.631990671157837
loss:3.5824859142303467
loss:3.5339303016662598
loss:3.4868125915527344
loss:3.4468159675598145
loss:3.4055981636047363
loss:3.3694119453430176
loss:3.334351062774658
loss:3.301457643508911
loss:3.271188259124756
loss:3.24688982963562
loss:3.2211949825286865
loss:3.1992430686950684
loss:3.1803269386291504
loss:3.1572985649108887
loss:3.1385133266448975
loss:3.116178274154663
loss:3.098071575164795
loss:3.0803802013397217
loss:3.060686826705932

In [9]:
#use trained network to generate poem
NEWRNN=CHARRNN(lenword=len(words)+1,n_chunk=n_chunk)
poem=NEWRNN.predict(words,word_nump_map,1,reuse=True)

INFO:tensorflow:Restoring parameters from C:\Users\jiema\Documents\tf-practice\charrnn.ckpt


In [10]:
print(poem)

奔日遏云声，觉来吟恶闻。入定彰烦逆，还惊亦直轻。潜妨卖药饵，自拟测馀生。地漫寒空翠，荣寒本是真。造化槎卢女，宸居况不飞。


In [4]:
print(x_batches[0])

[[   2   28  543  104  720    1    3 6109 6109 6109]
 [   2  649   48    9 2138    1    3 6109 6109 6109]
 [   2  424 4637 2126 1100    1    3 6109 6109 6109]
 [   2  345 1145 2601 2660    1    3 6109 6109 6109]
 [   2  821   10 1366  332    1    3 6109 6109 6109]
 [   2 1196   57  430   34  438    1    3 6109 6109]
 [   2    5   35  140  158  105    1    3 6109 6109]
 [   2  292 1461   18   56  298    1    3 6109 6109]
 [   2 1139  945 1320  522  661    1    3 6109 6109]
 [   2 1363  104   18   83  583    1    3 6109 6109]
 [   2  649   48    9 2138  392    1    3 6109 6109]
 [   2  168 2319 1625 1075 2319    1    3 6109 6109]
 [   2   64   28   47   64  739    1    3 6109 6109]
 [   2 1035  327   28  513  235    1    3 6109 6109]
 [   2  270 1237    9  358  807    1    3 6109 6109]
 [   2 5330  354   77  168  217    1    3 6109 6109]
 [   2  141    9   58 2329 2048    1    3 6109 6109]
 [   2  420  171  171   21  670 2816    1    3 6109]
 [   2 2254   44    9 2254  342    5    1    3